In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import os
os.chdir("../")
print(os.getcwd())
from parc.data import data_navier_stokes as data
from parc.model import model_ns as model
!nvidia-smi 

2024-05-27 12:00:01.411430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 12:00:02.055730: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/:/usr/local/cuda/include/:/usr/local/cuda/lib/:/usr/local/cuda/lib64/:/usr/lib/x86_64-linux-gnu/:/opt/apps/TensorRT-8.6.1.6/lib/:/opt/apps/TensorRT-8.6.1.6/include/:/usr/include/x86_64-linux-gnu/:/usr/include/
2024-05-27 12:00:02.056674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

/home/lweger/PARCv2
Mon May 27 12:00:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A30                     On  |   00000000:21:00.0 Off |                    0 |
| N/A   32C    P0             32W /  165W |     235MiB /  24576MiB |      2%      Default |
|                                         |                        |             Disabled |
+---------------------------

2024-05-27 12:00:02.780242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 12:00:02.947781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22280 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:21:00.0, compute capability: 8.0


# Data pipeline

In [2]:
Re_list = [7000,9000]    # all present data
train_list = [9000]    
test_list = [7000]

data_ns = data.DataNS(Re_list)
seq_input = data_ns.import_data(folder_path='./data/ns/')
seq_norm = data_ns.data_normalization(input_data=seq_input, no_of_channel=3)

train_seq, test_seq = data_ns.train_test_split(Re_list=Re_list, Re_test=test_list, Re_train=train_list,input_data=seq_norm[0])
train_data = data_ns.clip_data(input_seq=train_seq,no_of_fields=3,sequence_length=13)

# Training

### Stage 1: Differentiator training

In [3]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 798) 
dataset = dataset.batch(1)


### Stage 2: Integrator training (Can skip)

In [4]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "differentiator_training")
# parc.differentiator.load_weights('parc2_diff_ns_heun_10_2.h5')
# parc.poisson.load_weights('parc2_poisson_ns_heun_10_2.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

2024-05-27 12:00:49.565820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2024-05-27 12:00:51.689401: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7ef5c4d87a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-27 12:00:51.689422: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
2024-05-27 12:00:51.693768: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-27 12:00:51.786345: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26/26 [==============================] - 66s 514ms/step - total_loss: 9354.8867


In [5]:
parc.differentiator.save_weights('parc2_diff_TEST_ns_heun_12.h5')

In [6]:
parc.poisson.save_weights('parc2_poisson_TEST_ns_heun_12.h5')

In [7]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

26/26 [==============================] - 53s 395ms/step - total_loss: 7270.2393


In [8]:
parc.integrator.save_weights('parc2_int_TEST_ns_heun_12.h5')